<a href="https://colab.research.google.com/github/leeoos/sentence-classification/blob/main/src/HW1B_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1 B

## Set up

In [2]:
#@title modules installation

!pip install -q spacy
!python -m spacy download it_core_news_sm >> /dev/null

!pip install -q nltk

!pip install -q stanza

# if allowed
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 970.4/970.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 15.7 MB/s eta 0:00:00


In [3]:
#@title Imports

# general imports
import os
from tqdm import tqdm

# data manipulation
import json
import random
from pathlib import Path
from torchtext.vocab import Vocab, vocab
from collections import Counter, OrderedDict

# tokenization
import spacy
import nltk
from nltk.tokenize import WordPunctTokenizer
import stanza

# if allowed
from transformers import AutoTokenizer

# statistics
import matplotlib as plt

# torch
import torch
from torch import nn
from torch.nn import functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == 'cuda':
  !nvidia-smi


### Update some packages ###
!pip install -q conllu
!pip install -q torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.7 MB/s eta 0:00:00


## Preprocessing

In [4]:
#@title Get Data

if not os.path.exists("sentence-classification"):
  !git clone -q https://github.com/leeoos/sentence-classification.git

# set up paths
train_path = Path("sentence-classification/data/sentipolc16_task1_train_data.jsonl")
test_path = Path("sentence-classification/data/sentipolc16_task1_test_data.jsonl")

# look at some random samples from the train set
def sample_random_lines(filename, num_lines=10):
  with open(filename, 'r') as file:
    lines = file.readlines()
  # sample N random lines where N is num_lines or len(lines)
  return random.sample(lines, min(num_lines, len(lines)))

def print_random_json_lines(random_lines):
  for line in random_lines:
    try:
      json_data = json.loads(line.strip())
      print(json_data)
    except json.JSONDecodeError:
      print("Error decoding JSON from line:", line)

random_samples = sample_random_lines(train_path, 10)
print_random_json_lines(random_samples)

{'id': 195608613115543553, 'text': '@VodafoneIT Ho fatto un transloco e aspetto per il segnale al mio Vodafone station 65 giorni, ora devo pagare transloco e fattura. Opinioni?', 'topic': 'generico', 'choices': ['oggettivo', 'soggettivo'], 'label': 0}
{'id': 152367719310819328, 'text': "Il governo #Monti ce lo meritiamo per le gestione precedenti di un'#Italia che, per dirla alla Bocca, ha sempre perso le occasioni migliori", 'topic': 'politico', 'choices': ['oggettivo', 'soggettivo'], 'label': 1}
{'id': 393399050197676032, 'text': "ma davvero darren ha dedicato a mia 'any of those things?  ditemi di no........................................................", 'topic': 'generico', 'choices': ['oggettivo', 'soggettivo'], 'label': 1}
{'id': 134943542098399232, 'text': '@SBartezzaghi mario monti/rimontiamo è la prima volta che mi trovo in disaccordo con un anagramma', 'topic': 'politico', 'choices': ['oggettivo', 'soggettivo'], 'label': 1}
{'id': 134340487980134400, 'text': 'Mario Monti e

In [5]:
#@title Data Statistics

def statistics(path: Path) -> None:
  with open(path, 'r', encoding='utf-8') as file:
    num_lines = 0
    num_sogg = 0
    num_ogg = 0

    for _ in file:
      foo = json.loads(_)
      if foo["choices"][foo["label"]] == "oggettivo":
        num_ogg += 1
      else:
        num_sogg +=1
      num_lines += 1

    print(f"Total number of samples: {num_lines}")
    print(f"Oggetive samples: {round((num_ogg/num_lines)*100, 1)}%")
    print(f"Soggetive samples: {round((num_sogg/num_lines)*100, 1)}%")

print("Train-Set charateristics")
statistics(train_path)
print("\nTest-Set charateristics")
statistics(test_path)

Train-Set charateristics
Total number of samples: 7410
Oggetive samples: 31.2%
Soggetive samples: 68.8%

Test-Set charateristics
Total number of samples: 2000
Oggetive samples: 34.8%
Soggetive samples: 65.2%


In [6]:
#@title Language Dictionaries

nlp = spacy.load('it_core_news_sm')
nltk_nlp = WordPunctTokenizer()
stanza.download('it')
stanza_nlp = stanza.Pipeline(lang='it', processors='tokenize')

# if allowed
# bert_nlp = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
bert_nlp = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-xxl-cased")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: it (Italian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/it/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: it (Italian):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Done loading processors!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

In [7]:
#@title Tokenization

def get_tokenizer(tokenizer_type="spacy"):

  def custom_tokenizer(text):

    if tokenizer_type not in ["spacy", "nltk", "stanza", "bert"]:
      print(f"Error: invalid tokenizer type: {tokenizer_type}")
      return []

    if tokenizer_type == "spacy":
      doc = nlp(text)
      tokens = [token.text for token in doc]
      return tokens

    elif tokenizer_type == "nltk":
      """This tokenizer splits also the symbols inside the URL (links in genral)
      this may be a disadvantage"""
      tokens = nltk_nlp.tokenize(text)
      return tokens

    elif tokenizer_type == "stanza":
      doc = stanza_nlp(text)
      tokens = [token.text for sentence in doc.sentences for token in sentence.tokens]
      return tokens

    # if allowed
    elif tokenizer_type == "bert":
      tokens = bert_nlp.tokenize(text)
      token_ids = bert_nlp.convert_tokens_to_ids(tokens)
      return tokens, token_ids

    else:
      print("Error")
      return []

  return custom_tokenizer

# get a generic tokenizer
tokenizer = get_tokenizer

# test different tokenizer on the first  sentences of the train set
for line in random_samples:
  text = json.loads(line)["text"]
  print(f"Original sentence: {text}")
  spacy_list = tokenizer("spacy")(text)
  nltk_list = tokenizer("nltk")(text)
  stanza_list = tokenizer("stanza")(text)
  bert_list, _ = tokenizer("bert")(text)
  print(f"Num of tokens: {len(spacy_list)}", end="\t")
  print(f"Sapcy tokenization: {spacy_list}")
  print(f"Num of tokens: {len(nltk_list)}", end="\t")
  print(f"Nltk tokenization: {nltk_list}")
  print(f"Num of tokens: {len(stanza_list)}", end="\t")
  print(f"Stanza tokenization: {stanza_list}")
  print(f"Num of tokens: {len(bert_list)}", end="\t")
  print(f"Bert tokenization: {bert_list}")
  print()

Original sentence: @VodafoneIT Ho fatto un transloco e aspetto per il segnale al mio Vodafone station 65 giorni, ora devo pagare transloco e fattura. Opinioni?
Num of tokens: 26	Sapcy tokenization: ['@VodafoneIT', 'Ho', 'fatto', 'un', 'transloco', 'e', 'aspetto', 'per', 'il', 'segnale', 'al', 'mio', 'Vodafone', 'station', '65', 'giorni', ',', 'ora', 'devo', 'pagare', 'transloco', 'e', 'fattura', '.', 'Opinioni', '?']
Num of tokens: 27	Nltk tokenization: ['@', 'VodafoneIT', 'Ho', 'fatto', 'un', 'transloco', 'e', 'aspetto', 'per', 'il', 'segnale', 'al', 'mio', 'Vodafone', 'station', '65', 'giorni', ',', 'ora', 'devo', 'pagare', 'transloco', 'e', 'fattura', '.', 'Opinioni', '?']
Num of tokens: 28	Stanza tokenization: ['@', 'Vodafone', 'IT', 'Ho', 'fatto', 'un', 'transloco', 'e', 'aspetto', 'per', 'il', 'segnale', 'al', 'mio', 'Vodafone', 'station', '65', 'giorni', ',', 'ora', 'devo', 'pagare', 'transloco', 'e', 'fattura', '.', 'Opinioni', '?']
Num of tokens: 38	Bert tokenization: ['@', 'V

In [8]:
#@title Dataset Class

class Sentipolc16Dataset(Dataset):

  def __init__(
      self,
      input_file: Path,
      tokenizer_type: str = "spacy",
      device: str = "cuda",
      max_length : int = 1028
  ):
    # get a specific tokenizer
    tokenizer = get_tokenizer(tokenizer_type)

    # get samples from input file
    self.samples = []
    with open(input_file, "r") as file:
      # total_lines = sum(1 for _ in file)
      # iterator = tqdm(file, total=total_lines, desc="Processing", unit=" line")
      for line in tqdm(file, desc="Processing", unit=" line"):
        sample = json.loads(line.strip())
        # the sample dictionary contains the following key: idx, sentence and label (integer)
        sample["tokens"] = tokenizer(sample["text"])
        self.samples.append(sample)

    # set device
    self.device = torch.device(device)

    # set max sentence length allowed
    self.max_length = max_length

    # Initialize indexed data attribute but leave it None
    # Must be filled through the `.index(vocabulary, label_vocabulary)` method
    # each dictionary represents a sentence with two keys: "input_ids" and "label"
    self.indexed_data: list[dict] | None = None

    # Keep track of padding id
    self.padding_id: int | None = None


  def get_raw_element(self, idx: int) -> dict:
    """Utility function that gets the raw sample corresponding to `idx`."""
    return self.samples[idx]

  def __len__(self) -> int:
    return len(self.samples)

  def __getitem__(self, idx: int) -> dict:
    if self.indexed_data is None:
      raise RuntimeError(
        "Trying to retrieve samples but dataset has not been indexed yet!"
        + " Be sure to call `.index()` on this object."
        + " If you want to retrieve raw elements, call `.get_raw_elements(idx)"
      )
    return self.indexed_data[idx]

  def get_vocabulary(
    self,
    pad_token: str = "<pad>",
    unk_token: str = "<unk>",
    extra_tokens: list[str] = []
  ) -> Vocab:
    """Builds a `torchtext.vocab.Vocab` object from data stored in this object."""
    # most_common() returns a list of (token, count) pairs, so we convert them back into dictionary
    vocab_counter = dict(Counter(token for sent in self.samples for token in sent["tokens"]).most_common())
    # We build the vocabulary through a dictionary like {token: frequency, ...}
    vocabulary = vocab(vocab_counter, min_freq=1, specials=[pad_token, unk_token, *extra_tokens])
    # vocabulary(list of tokens) returns a list of values, so get the only one
    vocabulary.set_default_index(vocabulary([unk_token])[0])
    return vocabulary

  def set_padding_id(self, value: int) -> None:
    self.padding_id = value

  def index(self, vocabulary: Vocab) -> None:
    """Builds `self.indexed_data` by converting raw samples to input_ids following `vocabulary`"""
    if self.indexed_data is not None:
      print("Dataset has already been indexed. Keeping old index...")
    else:
      indexed_data = []
      for sample in self.samples:
        # append the dictionary containing ids of the input tokens and label
        indexed_data.append({"input_ids": vocabulary(sample["tokens"]), "label": sample["label"]})
      self.indexed_data = indexed_data

  def _collate_fn(self, raw_batch: list[dict]) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Batches together single elements of the dataset.
    This function:
    - converts each sentence made up of single input_ids to a padded tensor,
    - keeps track of the length of each sentence through `sequence_lengths`
    - builds a `labels` tensor storing the label for each sentence

    Args:
        raw_batch (list[dict]): a list of elements, as returned by the `__getitem__()` function.

    Returns:
        A tuple of three tensors, respectively `(sequence_lengths, padded_sequence, labels)`
    """
    if self.padding_id is None:
      raise RuntimeError("Padding value not set! Set it through .set_padding_id method.")

    # We need these sequence lengths to construct a `torch.nn.utils.rnn.PackedSequence` in the model
    sequence_lengths = torch.tensor([len(sample["input_ids"]) for sample in raw_batch], dtype=torch.long)
    padded_sequence = pad_sequence(
      (
        torch.tensor(sample["input_ids"], dtype=torch.long, device=self.device)
        for sample in raw_batch
      ),
      batch_first=True,
      padding_value=self.padding_id
    )
    labels = torch.tensor([sample["label"] for sample in raw_batch], device=self.device, dtype=torch.long)
    return sequence_lengths, padded_sequence, labels

In [9]:
# train_set = Sentipolc16Dataset(train_path)
# train_set.samples[88]

## Modelling

In [26]:
#@title Bidirectional LSTM

class BiLSTMModel(torch.nn.Module):

    def __init__(
        self,
        vocabulary_length: int,
        hidden_dim: int,
        bilstm_layers: int,
        bilstm_dropout: float,
        num_classes: int,
        padding_id: int,
        device: str = "cuda",
    ) -> None:
        super().__init__()

        # Prepare the device
        self.device = torch.device(device)
        print(self.device)

        # Embedding layer
        self.embedding = nn.Embedding(
            num_embeddings=vocabulary_length,
            embedding_dim=hidden_dim,
            padding_idx=padding_id, # avoid updating the gradient of padding entries
            device=self.device
        )

        # Feel free to modify and change any hyperparameters or architectural details

        # BiLSTM
        self.bilstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=bilstm_layers,
            batch_first=True,
            dropout=bilstm_dropout,
            bidirectional=True,
            device=self.device
        )

        # Projection layer --> this could be changed with a more complex logic
        self.projection = nn.Linear(
            in_features=hidden_dim * 2,
            out_features=num_classes,
            device=self.device
        )

    def forward(self, batch: tuple[torch.Tensor, torch.Tensor]) -> torch.Tensor:
        # Get the different parts of the batch
        sequence_lengths, input_ids = batch

        # First we embed the input tokens
        embeds = self.embedding(input_ids) # [B, S, H]
        # where B is the batch size, S is the sequence length and H is the hidden dimension

        # Pack the sequence to avoid gradient descent on padding tokens.
        # An alternative to packing sequences is using masking.
        packed = pack_padded_sequence(embeds, sequence_lengths, batch_first=True, enforce_sorted=False)

        # Then we pass it to the BiLSTM
        # The first output of the BiLSTM tuple, packed_output, is of size B x S x 2H,
        # where B is the batch size, S is the sequence length and H is the hidden dimension
        # hidden_state is of size [2 * num_layers, B, H], where the 2 is because we are using BiLSTMs instead of LSTMs.
        # cell_state has size [2 * num_layers, B, C] where C is the cell dimension of the internal LSTMCell.
        packed_output, (hidden_state, cell_state) = self.bilstm(packed)

        # We take the last two hidden representations of the BiLSTM (the second-to-last layer's output is forward; last
        # layer's is backward) by concatenating forward and backward over dimension 1.
        # Both tensors have shapes of [B, H], so concatenating them along the second dimension (dim 1) results in a new
        # tensor of shape [B, 2 * H]
        hidden = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)

        # Finally we project to the two final classes and return the logits of each class
        logits = self.projection(hidden) # [B, 2]
        return logits

In [41]:
#@title Trainer

class Trainer():
    """Utility class to train and evaluate a model."""

    def __init__(
        self,
        model: nn.Module,
        optimizer: torch.optim.Optimizer,
        log_steps: int = 1_000,
        log_level: int = 2
    ):
        self.model = model
        self.optimizer = optimizer
        self.loss_function = nn.CrossEntropyLoss() # this is the default loss used nearly everywhere in NLP

        self.log_steps = log_steps
        self.log_level = log_level

    def train(
        self,
        train_dataloader: DataLoader,
        # valid_dataloader: DataLoader,
        epochs: int = 1
    ) -> dict[str, list[float]]:
        """
        Args:
            train_dataloader: a DataLoader instance containing the training instances.
            valid_dataloader: a DataLoader instance used to evaluate learning progress.
            epochs: the number of times to iterate over train_dataset.

        Returns:
            avg_train_loss: the average training loss on train_dataset over epochs.
        """
        assert epochs >= 1 and isinstance(epochs, int)
        if self.log_level > 0:
            print('Training ...')
        train_loss = 0.0

        losses = {
            "train_losses": [],
            # "valid_losses": [],
            "valid_acc": [],
        }

        for epoch in range(1, epochs + 1):
            if self.log_level > 0:
                print(' Epoch {:2d}'.format(epoch))

            epoch_loss = 0.0
            self.model.train()
            # print(self.model.device)

            # for each batch
            for step, (sequence_lengths, inputs, labels) in enumerate(train_dataloader):
                self.optimizer.zero_grad()

                # We get the predicted logits from the model, with no need to perform any flattening
                # as both predictions and labels refer to the whole sentence.
                # print(inputs.device)
                # print(sequence_lengths.device)
                # break
                predictions = self.model((sequence_lengths, inputs))

                # The CrossEntropyLoss expects the predictions to be logits, i.e. non-softmaxed scores across
                # the number of classes, and the labels to be a simple tensor of labels.
                # Specifically, predictions needs to be of shape [B, C], where B is the batch size and C is the number of
                # classes, while labels must be of shape [B] where each element l_i should 0 <= l_i < C.
                # See https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html for more information.
                sample_loss = self.loss_function(predictions, labels)
                sample_loss.backward()
                self.optimizer.step()

                epoch_loss += sample_loss.cpu().tolist()

                if self.log_level > 1 and (step % self.log_steps) == (self.log_steps - 1):
                    print('\t[E: {:2d} @ step {}] current avg loss = {:0.4f}'.format(epoch, step, epoch_loss / (step + 1)))

            avg_epoch_loss = epoch_loss / len(train_dataloader)

            if self.log_level > 0:
                print('\t[E: {:2d}] train loss = {:0.4f}'.format(epoch, avg_epoch_loss))

            # valid_loss, valid_acc = self.evaluate(valid_dataloader)

            losses["train_losses"].append(avg_epoch_loss)
            # losses["valid_losses"].append(valid_loss)
            # losses["valid_acc"].append(valid_acc)

            if self.log_level > 0:
                # print('  [E: {:2d}] valid loss = {:0.4f}, valid acc = {:0.4f}'.format(epoch, valid_loss, valid_acc))
                ...

        if self.log_level > 0:
            print('... Done!')

        return losses

    def _compute_acc(self, logits: torch.Tensor, labels: torch.Tensor) -> float:
        # logits [B, 2] are the logits outputted by the BiLSTM model's forward()
        # We take the argmax along the second dimension (dim=1), so we get a tensor of shape [B]
        # where each element is 0 if the 0-class had higher logit, 1 otherwise.
        predictions = torch.argmax(logits, dim=1)
        # We can then directly compare each prediction with the labels, as they are both tensors with shape [B].
        # The average of the boolean equality checks between the two is the accuracy of these predictions.
        # For example, if:
        #   predictions = [1, 0, 0, 1, 1]
        #   labels = [1, 0, 1, 1, 1]
        # The comparison is:
        #   (predictions == labels) => [1, 1, 0, 1, 1]
        # which averaged gives an accuracy of 4/5, i.e. 0.80.
        return torch.mean((predictions == labels).float()).tolist() # type: ignore

    def evaluate(self, valid_dataloader: DataLoader) -> tuple[float, float]:
        """
        Args:
            valid_dataloader: the DataLoader to use to evaluate the model.

        Returns:
            avg_valid_loss: the average validation loss over valid_dataloader.
        """
        valid_loss = 0.0
        valid_acc = 0.0
        # When running in inference mode, it is required to have model.eval() AND .no_grad()
        # Among other things, these set dropout to 0 and turn off gradient computation.
        self.model.eval()
        with torch.no_grad():
            for batch in valid_dataloader:
                sequence_lengths, inputs, labels = batch

                logits = self.model((sequence_lengths, inputs))

                # Same considerations as the training step apply here
                sample_loss = self.loss_function(logits, labels)
                valid_loss += sample_loss.tolist()

                sample_acc = self._compute_acc(logits, labels)
                valid_acc += sample_acc

        return valid_loss / len(valid_dataloader), valid_acc / len(valid_dataloader),

    def predict(self, batch: tuple[torch.Tensor, torch.Tensor]) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Args:
            x: a tensor of indices
        Returns:
            A tuple composed of:
            - the logits of each class, 0 and 1
            - the prediction for each sample in the batch
              0 if the sentiment of the sentence is negative, 1 if it is positive.
        """
        self.model.eval()
        with torch.no_grad():
            sequence_lengths, inputs = batch
            logits = self.model(sequence_lengths, inputs) # [B, 2]
            predictions = torch.argmax(logits, -1) # [B, 1] computed on the last dimension of the logits tensor
            return logits, predictions

In [17]:
pad_token, unk_token = "<pad>", "<unk>"
device = "cuda"

train_dataset = Sentipolc16Dataset(train_path, device=device)
test_dataset = Sentipolc16Dataset(test_path, device=device)

vocabulary = train_dataset.get_vocabulary(pad_token=pad_token, unk_token=unk_token)
padding_id = vocabulary([pad_token])[0]

train_dataset.set_padding_id(padding_id)
test_dataset.set_padding_id(padding_id)

train_dataset.index(vocabulary)
test_dataset.index(vocabulary)

print(f"Training len: {len(train_dataset)}")
print(f"Test len: {len(test_dataset)}")

np.save('train',train_dataset, allow_pickle=True)
np.save('test',test_dataset, allow_pickle=True)




Processing: 7410 line [01:15, 97.90 line/s] 
Processing: 2000 line [00:15, 130.48 line/s]


Training len: 7410
Test len: 2000


In [20]:
import numpy as np

In [23]:
loaded_train_dataset = np.load("train.npy")
loaded_test_dataset = np.load("test.npy")
training_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=train_dataset._collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False, collate_fn=test_dataset._collate_fn)

In [42]:
sentiment_tagger = BiLSTMModel(
    vocabulary_length=len(vocabulary),
    hidden_dim=64,
    bilstm_layers=2,
    bilstm_dropout=0.3,
    num_classes=2,
    padding_id=padding_id,
    device=device
)

cuda


In [13]:
print(train_dataset.__getitem__(1))
print(train_dataset.indexed_data[1])
train_dataset.index(vocabulary)
print(vocabulary["<pad>"])

{'input_ids': [2110, 1577, 4, 1578, 553, 22, 5, 7155, 63, 4415], 'label': 1}
{'input_ids': [2110, 1577, 4, 1578, 553, 22, 5, 7155, 63, 4415], 'label': 1}
Dataset has already been indexed. Keeping old index...
0


In [43]:
trainer = Trainer(
    model=sentiment_tagger,
    optimizer=torch.optim.Adam(sentiment_tagger.parameters(), lr=0.0001),
    log_steps=100
)

In [44]:
losses = trainer.train(training_dataloader, epochs=10)

Training ...
 Epoch  1
	[E:  1] train loss = 0.6780
 Epoch  2
	[E:  2] train loss = 0.6171
 Epoch  3
	[E:  3] train loss = 0.5974
 Epoch  4
	[E:  4] train loss = 0.5881
 Epoch  5
	[E:  5] train loss = 0.5789
 Epoch  6
	[E:  6] train loss = 0.5696
 Epoch  7
	[E:  7] train loss = 0.5606
 Epoch  8
	[E:  8] train loss = 0.5539
 Epoch  9
	[E:  9] train loss = 0.5462
 Epoch 10
	[E: 10] train loss = 0.5381
... Done!


## Baselines

- majority baseline  to spot imbalance dataset
- random baseline
- custom baseline (rnn not lstm???)